# Setup

In [71]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
# For colab
if IN_COLAB:
  import sys
  !git clone https://github.com/elements72/EDiReF-subtask-III.git
  %pip install lightning
  %pip install wandb
  sys.path.append('./EDiReF-subtask-III')
  %cd ./EDiReF-subtask-III
  num_workers = 2
else:
  num_workers = 0
  pass

In [62]:
import logging
log = logging.getLogger("pytorch_lightning")
log.propagate = False
log.setLevel(logging.ERROR)

In [63]:
# Reload modules, do not use cache
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [64]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import numpy as np
from utils import train_model, hyperparameters_tuning, train_model_seeds
import torch
from baselines import random_metrics, majority_metrics
from baselines import BertBaseline
from baselines import RandomUniformClassifier
from baselines import MajorityClassifier
from metrics import F1ScoreCumulative, F1ScoreDialogues
import torch
from sklearn.model_selection import train_test_split
from attention import DialoguesAttention
from lstm_model import LSTMResModel
from dataset import MeldDataModule


In [65]:
import wandb
wandb.login()

True

# Dataset

In [66]:
batch_size = 8
# Create and setup data module
datamodule = MeldDataModule(batch_size=batch_size, num_workers=num_workers)
datamodule.prepare_data()
datamodule.setup()
# Get dataframes
train_data = datamodule.train_data
val_data = datamodule.val_data
test_data = datamodule.test_data
# Get encoder
emotion_encoder = datamodule.emotion_encoder
#Base Dataloaders
train_loader = datamodule.train_dataloader(collate_context=False, batch_size=batch_size)
val_loader = datamodule.val_dataloader(collate_context=False, batch_size=batch_size)
test_loader = datamodule.test_dataloader(collate_context=False, batch_size=batch_size)

In [67]:
from sklearn.utils.class_weight import compute_class_weight

class_weights_emotion = compute_class_weight('balanced', classes=np.unique(train_data['emotions'].explode()),
                                             y=train_data['emotions'].explode())
class_weights_emotion = torch.tensor(class_weights_emotion, dtype=torch.float)

class_weights_trigger = compute_class_weight('balanced', classes=np.unique(train_data['triggers'].explode()), 
                                             y=train_data['triggers'].explode())
class_weights_trigger = torch.tensor(class_weights_trigger, dtype=torch.float)

class_weights_emotion

tensor([1.3135, 4.8692, 4.3329, 0.7756, 0.3249, 2.0598, 1.0525])

In [68]:
def get_hyperparams(model_name, freeze, lr):
    hyperparameters = {
        "lr": lr,
        "class_weights_emotion": class_weights_emotion,
        "class_weights_trigger": class_weights_trigger,
        "hidden_layers": 2,
        "use_encoder_cache": True if freeze else False,
        "encoder_cache_size": 100000,
        "freeze_bert": freeze,
        "dropout": 0.2,
        "bert_model_name": "bert-base-uncased",
        }
    if "lstm" in model_name:
        lstm_kwargs = {
            "hidden_size": 128,
            "num_layers": 2,
            "bidirectional": False,
        }
        hyperparameters["lstm_kwargs"] = lstm_kwargs
    if "emoberta" in model_name:
        hyperparameters["bert_model_name"] = "roberta-large"
    return hyperparameters


In [86]:
lr = 1e-3
epochs = 20
tune_hyperparameters = False
seeds = [6, 9, 42, 150, 300]

batch_size_freeze = 32
batch_size_unfreeze = 8

train_loader_freeze = datamodule.train_dataloader(collate_context=False, batch_size=batch_size_freeze )
val_loader_freeze = datamodule.val_dataloader(collate_context=False, batch_size=batch_size_freeze   )
test_loader_freeze = datamodule.test_dataloader(collate_context=False, batch_size=batch_size_freeze  )

train_loader_unfreeze = datamodule.train_dataloader(collate_context=False, batch_size=batch_size_unfreeze)
val_loader_unfreeze = datamodule.val_dataloader(collate_context=False, batch_size=batch_size_unfreeze)
test_loader_unfreeze = datamodule.test_dataloader(collate_context=False, batch_size=batch_size_unfreeze)

train_loader_emoberta = datamodule.train_dataloader(collate_context=True, batch_size=batch_size_unfreeze)
val_loader_emoberta = datamodule.val_dataloader(collate_context=True, batch_size=batch_size_unfreeze)
test_loader_emoberta = datamodule.test_dataloader(collate_context=True, batch_size=batch_size_unfreeze)


models = {
    "bert-baseline-freeze":{'model_class': BertBaseline, 'hyperparameters': get_hyperparams("bert-baseline-freeze", freeze=True, lr=lr), "train_loader": train_loader_freeze, "val_loader": val_loader_freeze},
    "bert-baseline-unfreeze":{'model_class': BertBaseline, 'hyperparameters': get_hyperparams("bert-baseline-unfreeze", freeze=False, lr=lr), "train_loader": train_loader_unfreeze, "val_loader": val_loader_unfreeze},
    "bert-lstm-freeze":{'model_class': LSTMResModel, 'hyperparameters': get_hyperparams("bert-lstm-freeze", freeze=True, lr=lr), "train_loader": train_loader_freeze, "val_loader": val_loader_freeze},
    "bert-lstm-unfreeze":{'model_class': LSTMResModel, 'hyperparameters': get_hyperparams("bert-lstm-unfreeze", freeze=False, lr=lr), "train_loader": train_loader_unfreeze, "val_loader": val_loader_unfreeze},
    "bert-attention-freeze":{'model_class': DialoguesAttention, 'hyperparameters': get_hyperparams("bert-attention-freeze", freeze=True, lr=lr), "train_loader": train_loader_freeze, "val_loader": val_loader_freeze},
    "bert-attention-unfreeze":{'model_class': DialoguesAttention, 'hyperparameters': get_hyperparams("bert-attention-unfreeze", freeze=False, lr=lr), "train_loader": train_loader_unfreeze, "val_loader": val_loader_unfreeze},
    "emoberta": {'model_class': BertBaseline, 'hyperparameters': get_hyperparams("emoberta", freeze=True, lr=lr), "train_loader": train_loader_emoberta, "val_loader": val_loader_emoberta},
    }

# to_train = ['emoberta']
    
# for model_name, model_dict in models.items():
#   if model_name not in to_train:
#     continue
#   train_model_seeds(**model_dict, model_name=model_name, seeds=seeds, epochs=epochs, wandb=wandb)

In [90]:
from utils import load_model, evaluate_model
import os
results_path = Path("results")
results = pd.DataFrame()
# Check if file exists
if os.path.isfile(results_path / "results.csv"):
    results = pd.read_csv(results_path / "results.csv", index_col=0)
else:
    for model_name, model_dict in models.items():
        for seed in seeds:
            model = load_model(model_dict["model_class"], f"{model_name}-seed-{seed}")
            result = evaluate_model(model, model_name, test_loader_unfreeze, test=True)
            # concat results
            df = pd.concat([df, results])
    # save results
    results.to_csv(results_path / "results.csv", index=True)

/home/elements72/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/migration/utils.py:55: The loaded checkpoint was produced with Lightning v2.1.3, which is newer than your current Lightning version: v2.1.0


Using cache of size 100000 for the embedding


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/elements72/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃           Test metric            ┃           DataLoader 0           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    f1_test_cumulative_emotion    │        0.3755950927734375        │
│    f1_test_cumulative_trigger    │       0.28153717517852783        │
│ f1_test_cumulative_trigger_multi │       0.46300917863845825        │
│    f1_test_dialogues_emotion     │       0.23318016529083252        │
│    f1_test_dialogues_trigger     │        0.3105781078338623        │
│ f1_test_dialogues_trigger_multi  │       0.46484196186065674        │
│        test_emotion_loss         │        1.5625115633010864        │
│            test_loss             │        2.237989902496338         │
│        test_trigger_loss         │        0.6754782795906067        │
└──────────────────────────────────┴──────────────────────────────────┘

...

/home/elements72/.local/lib/python3.10/site-packages/lightning/pytorch/utilities/migration/utils.py:55: The loaded checkpoint was produced with Lightning v2.1.3, which is newer than your current Lightning version: v2.1.0


Using cache of size 100000 for the embedding


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/elements72/.local/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacty of 1.95 GiB of which 28.38 MiB is free. Including non-PyTorch memory, this process has 1.92 GiB memory in use. Of the allocated memory 1.65 GiB is allocated by PyTorch, and 231.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [15]:
# Random Classifier
# Test set
random_uniform_classifier = RandomUniformClassifier(6, 1)
metrics_random, emotions_predictions, triggers_predictions= random_metrics(random_uniform_classifier, test_loader, num_classes_emotions=7, num_classes_triggers=2)

print('Test set:')
metrics_random.head()

Test set:


,Emotions,Triggers Binary,Triggers Multiclass
F1 Cumulative,0.110378,0.0,0.460621
F1 Dialogues,0.086337,0.0,0.451750


In [16]:
# Random Classifier
# Validation set
metrics_random_val, emotions_predictions_val, triggers_predictions_val= random_metrics(random_uniform_classifier, val_loader, num_classes_emotions=7, num_classes_triggers=2)

print('Validation set:')
metrics_random_val.head()

Validation set:


,Emotions,Triggers Binary,Triggers Multiclass
F1 Cumulative,0.113110,0.0,0.440633
F1 Dialogues,0.089383,0.0,0.426301


In [17]:
# Majority Classifier
# Test set
majority_classifier = MajorityClassifier(num_classes_emotions=6, num_classes_triggers=1, padding_value_emotions=7, padding_value_triggers=2)
majority_classifier.fit(train_loader)
metrics_majority, emotions_predictions, triggers_predictions = majority_metrics(majority_classifier, test_loader, num_classes_emotions=7, num_classes_triggers=2)

print('Test set:')
metrics_majority.head()

Test set:


,Emotions,Triggers Binary,Triggers Multiclass
F1 Cumulative,0.084635,0.0,0.460621
F1 Dialogues,0.177654,0.0,0.451750


In [18]:
# Majority Classifier
# Validation set
metrics_majority_val, emotions_predictions_val, triggers_predictions_val = majority_metrics(majority_classifier, val_loader, num_classes_emotions=7, num_classes_triggers=2)

print('Validation set:')
metrics_majority_val.head()

Validation set:


,Emotions,Triggers Binary,Triggers Multiclass
F1 Cumulative,0.085029,0.0,0.440633
F1 Dialogues,0.183634,0.0,0.426301
